# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,18.09,84,3,5.11,ZA,1701896718
1,1,whitehorse,60.7161,-135.0538,-1.57,86,75,6.17,CA,1701896494
2,2,ribeira grande,38.5167,-28.7000,18.22,72,40,10.29,PT,1701896719
3,3,ilulissat,69.2167,-51.1000,-5.88,75,100,3.48,GL,1701896720
4,4,khash,28.2211,61.2158,9.60,28,7,0.74,IR,1701896720


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,urangan,-25.3000,152.9000,23.80,69,0,4.15,AU,1701896741
91,91,uvalde,29.3669,-99.7670,21.29,40,0,3.09,US,1701896608
146,146,tabou,4.4230,-7.3528,26.51,81,0,3.50,CI,1701896531
157,157,gao,16.6362,1.6370,25.37,13,0,4.28,ML,1701896782
261,261,as suwayq,23.8494,57.4386,24.18,47,0,2.07,OM,1701896814
300,300,gwadar,25.1216,62.3254,23.77,41,0,1.22,PK,1701896826
377,377,calvinia,-31.4707,19.7760,22.34,24,0,1.72,ZA,1701896852
432,432,camalu,30.8500,-116.0667,21.88,46,0,4.35,MX,1701896872
478,478,binkolo,8.9533,-11.9779,24.79,71,0,0.38,SL,1701896897
498,498,illapel,-31.6308,-71.1653,25.61,33,0,3.55,CL,1701896611


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity


hotel_df = city_data_df[["City",
                        "Country",
                        "Lat",
                        "Lng",
                        "Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\18607\AppData\Local\Temp\ipykernel_31304\3650545846.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
60,urangan,AU,-25.3000,152.9000,69,
91,uvalde,US,29.3669,-99.7670,40,
146,tabou,CI,4.4230,-7.3528,81,
157,gao,ML,16.6362,1.6370,13,
261,as suwayq,OM,23.8494,57.4386,47,
300,gwadar,PK,25.1216,62.3254,41,
377,calvinia,ZA,-31.4707,19.7760,24,
432,camalu,MX,30.8500,-116.0667,46,
478,binkolo,SL,8.9533,-11.9779,71,
498,illapel,CL,-31.6308,-71.1653,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
      "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{lon},{lat},{radius}"

    params["bias"] =  f"proximity:{lon},{lat}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
urangan - nearest hotel: Ramada
uvalde - nearest hotel: No hotel found
tabou - nearest hotel: hôtel le rochet
gao - nearest hotel: No hotel found
as suwayq - nearest hotel: No hotel found
gwadar - nearest hotel: Sadaf Resorts
calvinia - nearest hotel: No hotel found
camalu - nearest hotel: Margaritaville
binkolo - nearest hotel: R&B Hotel
illapel - nearest hotel: Domingo Ortiz de Rosas


,City,Country,Lat,Lng,Humidity,Hotel Name
60,urangan,AU,-25.3000,152.9000,69,Ramada
91,uvalde,US,29.3669,-99.7670,40,No hotel found
146,tabou,CI,4.4230,-7.3528,81,hôtel le rochet
157,gao,ML,16.6362,1.6370,13,No hotel found
261,as suwayq,OM,23.8494,57.4386,47,No hotel found
300,gwadar,PK,25.1216,62.3254,41,Sadaf Resorts
377,calvinia,ZA,-31.4707,19.7760,24,No hotel found
432,camalu,MX,30.8500,-116.0667,46,Margaritaville
478,binkolo,SL,8.9533,-11.9779,71,R&B Hotel
498,illapel,CL,-31.6308,-71.1653,33,Domingo Ortiz de Rosas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)